In [9]:
# Widgets
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

# Vamos usar o pandas para uma tabela
import pandas as pd

# Vamos fazer o gráfico das funções, por isso precisamos usar o módulo matplotlib.
# Vamos também escolher o modo de exibição do gráfico. No nosso caso, para simplificar, escolhemos
%matplotlib inline
# Em seguida, importamos apenas o submódulo pyplot da matplotlib com o "apelido" plt:
import matplotlib.pyplot as plt

# Por fim, importamos algumas facilidades para a exibição de LaTeX e outras saídas
from IPython.display import display, clear_output, Markdown, Latex

# Importaremos o numpy para podermos fazer corretamente o gráfico das funções.
import numpy as np

# Quantos minutos?

Na Cidade de Tecnópolis, existem duas operadoras de telefonia celular. "MundoFone" oferece um plano mensal com uma tarifa de R\\$35 por 120 minutos, mais R\\$0,10 por cada minuto adicional. "UniverCell" oferece um plano mensal com uma tarifa de R\\$30 por 100 minutos, e cada minuto adicional custa R\\$0,15 até um máximo de 300 minutos. Depois disso, cada minuto extra custa R\\$0,05. 

Quantos minutos você deveria utilizar por mês para que "MundoFone" seja a melhor opção, isto é, custe menos que "UniverCell"?

## Preparando o problema (inspeção)

Primeiramente, vamos olhar para como calcular a franquia mensal para cada operadora. Normalmente, quando olhamos para problemas assim, escolhemos alguns valores ao acaso e calculamos o resultado. Vamos tentar uma tabela:

In [13]:
def mundofone(minutos):
    if minutos <= 120:
        custo = 35
    else:
        custo = 35+(minutos-120)*0.1
    return custo

In [14]:
def univercell(minutos):
    if minutos <= 100:
        custo = 30
    elif minutos <= 300:
        custo = 30 + (minutos-100)*0.15
    else:
        custo = 30 + 200*0.15 + (minutos-300)*0.05
    return custo

In [20]:
minutos_input = widgets.Text(
                    value='',
                    placeholder='Entre minutos (decimal)',
                    description="Minutos =",
)

# Inicializar tabela com a primeira linha
dados = [['Tarifa Básica', 35, 30]] 
tabela = pd.DataFrame(dados, columns = ['Minutos', 'MundoFone', 'UniverCell']) 

interact_manual.opts['manual_name'] = 'Calcular Custo'

@interact_manual(minutos=minutos_input, tabela=fixed(tabela))
def atualizar_tabela(minutos, tabela):
    if minutos != '':
        tempo = float(minutos)
        if len(tabela[tabela['Minutos'] == tempo]) > 0:
            # este valor já foi calculado; não faça nada!
            pass
        else:
            tabela.loc[len(tabela)+1] = (tempo, mundofone(tempo), univercell(tempo))
    # eliminar o índice para simplificar a tabela
    tabela.index = ['']*len(tabela)
    display(tabela)

interactive(children=(Text(value='', description='Minutos =', placeholder='Entre minutos (decimal)'), Button(d…

Agora, seria interessante ver como esses valores sem comparam em um gráfico. Vamos marcar, os valores calculados acima da seguinte forma: o tempo será marcado no eixo horizontal, e o custo de cada companhia será marcado no eixo vertical.

In [29]:
# Vamos gerar uma janela de gráficos usando a biblioteca matplotlib.pyplot - que apelidamos de plt
fig, ax = plt.subplots()

plt.close(fig)

botao_pontos = widgets.Button(
                    description='Mostrar valores calculados no gráfico',
                    disabled=False,
                    button_style='',
                    tooltip='Mostrar valores calculados no gráfico',
                    layout=widgets.Layout(width='auto')
                    )

botao_conectar = widgets.Button(
                    description='Conectar pontos',
                    disabled=False,
                    button_style='',
                    tooltip='Conectar pontos',
                    layout=widgets.Layout(width='auto')
                    )

display(widgets.HBox([botao_pontos, botao_conectar]))
saida_graficos = widgets.Output()
display(widgets.HBox(children=(saida_graficos,)))

def graficar_pontos(grafico):
    ax.clear()
    ax.plot(tabela['Minutos'][1:], tabela['MundoFone'][1:], 'r*', label="MundoFone")
    ax.plot(tabela['Minutos'][1:], tabela['UniverCell'][1:], 'bo', label="UniverCell")
    ax.legend()
    with saida_graficos:
        clear_output(wait=True)
        display(fig)
        
def conectar_pontos(conectar):
    ax.clear()
    entradas = tabela[1:].sort_values(by='Minutos')
    ax.plot(entradas['Minutos'], entradas['MundoFone'], 'r*')
    ax.plot(entradas['Minutos'], entradas['UniverCell'], 'bo')
    ax.plot(entradas['Minutos'], entradas['MundoFone'], 'r')
    ax.plot(entradas['Minutos'], entradas['UniverCell'], 'b')
    ax.legend(("MundoFone", "UniverCell"))
    with saida_graficos:
        clear_output(wait=True)
        display(fig)

botao_pontos.on_click(graficar_pontos)
botao_conectar.on_click(conectar_pontos)

Agora, podemos ver que existe um intervalo em que os pontos (e a linha) vermelhos ficam abaixo dos pontos azuis; isso significa que MundoFone é mais barata que UniverCell somente nesse intervalo. Vamos ver quando isso ocorre.

Do gráfico, podemos ver que o intervalo em que estamos interessados é determinado pelos dois pontos em que as linhas azul e vermelha se cruzam. Agora, como podemos encontrar estes pontos?

## Encontrando a intersecção das duas linhas

Ambas linhas são determinadas por *funções*, que são maneiras de conectar os minutos aos custos correspondentes em cada companhia de telefonia. Apenas para simplificar, vamos dar um apelido ao valor do custo mensal de cada companhia. O custo para MundoFone será chamado $W$ e o custo para UniverCell será chamado $U$. 

Podemos ver que ambos os custos são determinados por quantos minutos são usados em cada mês. Podemos então dizer coisas assim:

In [30]:
tabela.set_index('Minutos')
for indice, linha in tabela[1:].iterrows():
    print('Para {0} minutos, temos W({0}) = {1}, U({0}) = {2}.'.format(int(linha[0]), linha[1], linha[2]))

Para 100 minutos, temos W(100) = 35.0, U(100) = 30.0.
Para 120 minutos, temos W(120) = 35.0, U(120) = 33.0.
Para 200 minutos, temos W(200) = 43.0, U(200) = 45.0.
Para 150 minutos, temos W(150) = 38.0, U(150) = 37.5.
Para 300 minutos, temos W(300) = 53.0, U(300) = 60.0.
Para 350 minutos, temos W(350) = 58.0, U(350) = 62.5.
Para 250 minutos, temos W(250) = 48.0, U(250) = 52.5.
Para 400 minutos, temos W(400) = 63.0, U(400) = 65.0.
Para 500 minutos, temos W(500) = 73.0, U(500) = 70.0.
Para 450 minutos, temos W(450) = 68.0, U(450) = 67.5.


Note que para valores que não estão na tabela, não podemos decidir diretamente quanto isso custaria, a não ser que calculemos este número também. Felizmente, existe uma maneira mais fácil de descrever o custo total sem termos que listar todos as possíveis quantidades de minutos: usando a *lei* da função. Em nosso caso, tivemos o seguinte: vamos chamar $t$ a quantidade total de minutos usada naquele mês. Então,

In [33]:
# Título
display(Markdown('Para MundoFone:\n (Pressione enter após entrar com o valor)'))

# Mostrar nome da função - W(t)
nome_mundofone = widgets.Output(layout=widgets.Layout(width='80px'))
with nome_mundofone:
    display(Latex('$W(t)= $'))
    
# Mostrar primeiro intervalo para a definição da função W
condicao1_mundofone = widgets.Output(layout=widgets.Layout(width='auto', grid_area='condicao1_mundofone'))
with condicao1_mundofone:
    display(Latex('para $t \leq 120$'))
    
# Mostrar segundo intervalo para a definição da função W
condicao2_mundofone = widgets.Output(layout=widgets.Layout(width='auto', grid_area='condicao2_mundofone'))
with condicao2_mundofone:
    display(Latex('para $t > 120$.'))

# Mostrar gráfico resultante
fig_mundofone, ax_mundofone = plt.subplots()
grafico_mundofone = widgets.Output()
plt.close(fig_mundofone)

def preparar_fig_mundofone(fig_mundofone, ax_mundofone, grafico_mundofone):
    with grafico_mundofone:
        entradas = np.linspace(0,500,50)
        saidas_mundofone = []
        for t in entradas:
            saidas_mundofone.append(mundofone(t))
        ax_mundofone.plot(entradas, saidas_mundofone, 'r', label='MundoFone')
        ax_mundofone.legend()
        display(fig_mundofone)

preparar_fig_mundofone(fig_mundofone, ax_mundofone, grafico_mundofone)
        
# Widgets para o input das leis da função
caixa1_mundofone = widgets.Text(layout=widgets.Layout(width='150px', grid_area='caixa1_mundofone'))
caixa2_mundofone = widgets.Text(layout=widgets.Layout(width='150px', grid_area='caixa2_mundofone'))

# Widgets são mostrados em uma tabela
grid_mundofone = widgets.GridBox(
    children=[caixa1_mundofone, condicao1_mundofone, caixa2_mundofone, condicao2_mundofone],
    layout=widgets.Layout(
            width='350px',
            grid_template_rows='auto auto',
            grid_template_columns='150px 200px',
            grid_template_areas='''
                            "caixa1_mundofone condicao1_mundofone"
                            "caixa2_mundofone condicao2_mundofone"
                            ''')
            )

# Este widget contém os três widgets definidos acima, lado a lado.
caixa_horizontal = widgets.HBox(children=[nome_mundofone, grid_mundofone, grafico_mundofone],
                    layout=widgets.Layout(width='90%', align_items='center', justify_content='center'))

def grafico_mundofone_condicao1(caixa):
    entradas = np.linspace(0,120,50)
    valores = []
    for t in entradas:
        # atenção com eval!
        valores.append(eval(caixa.value))
    ax_mundofone.plot(entradas, valores, 'k-', label="Meu gráfico ($0\leq t\leq 120$)")
    ax_mundofone.legend()
    with grafico_mundofone:
        clear_output(wait=True)
        display(fig_mundofone)
        
def grafico_mundofone_condicao2(caixa):
    entradas = np.linspace(120,500,50)
    valores = []
    for t in entradas:
        valores.append(eval(caixa.value))
    ax_mundofone.plot(entradas, valores, 'k-', label="Meu gráfico ($t>120$)")
    ax_mundofone.legend()
    with grafico_mundofone:
        clear_output(wait=True)
        display(fig_mundofone)
    
display(caixa_horizontal)
caixa1_mundofone.on_submit(grafico_mundofone_condicao1)
caixa2_mundofone.on_submit(grafico_mundofone_condicao2)

botao_limpar_mundofone = widgets.Button(
                            description='Limpar gráfico',
                            #disabled=False,
                            button_style='',
                            tooltip='Limpar gráfico',
                            layout=widgets.Layout(width='auto')
                            )

def limpar_grafico_mundofone(limpar):
    with grafico_mundofone:
        clear_output(wait=True)
        ax_mundofone.clear()
        preparar_fig_mundofone(fig_mundofone, ax_mundofone, grafico_mundofone)

display(botao_limpar_mundofone)
botao_limpar_mundofone.on_click(limpar_grafico_mundofone)

Para MundoFone:
 (Pressione enter após entrar com o valor)

Button(description='Limpar gráfico', layout=Layout(width='auto'), style=ButtonStyle(), tooltip='Limpar gráfico…

Let's repeat this for CellUniverse (remember that here we have 3 price ranges):

In [9]:
# Title
display(Markdown('For CellUniverse:'))

# Display function name - U(t)
name_cu = widgets.Output(layout=widgets.Layout(width='80px'))
with name_cu:
    display(Latex('$U(t)= $'))
    
# Display first interval for the function definition
cond1_cu = widgets.Output(layout=widgets.Layout(width='auto', grid_area='cond1_cu'))
with cond1_cu:
    display(Latex('for $t \leq 100$'))
    
# Display second interval for the function definition
cond2_cu = widgets.Output(layout=widgets.Layout(width='auto', grid_area='cond2_cu'))
with cond2_cu:
    display(Latex('for $100 < t \leq 300$'))

# Display third interval for the function definition
cond3_cu = widgets.Output(layout=widgets.Layout(width='auto', grid_area='cond3_cu'))
with cond3_cu:
    display(Latex('for $t > 300$.'))

# Display resulting function graph
fig_cu, ax_cu = plt.subplots()
plt.close(fig_cu)
outgraph_cu = widgets.Output()

def start_fig_cu(fig_cu, ax_cu, outgraph_cu):
    with outgraph_cu:
        values = np.linspace(0,500,50)
        cu = []
        for t in values:
            cu.append(celluniverse(t))
        ax_cu.plot(values, cu, 'b', label='CellUniverse')
        ax_cu.legend()
        display(fig_cu)

start_fig_cu(fig_cu, ax_cu, outgraph_cu)
    
# Widgets for the function law input
box1_cu = widgets.Text(layout=widgets.Layout(width='150px', grid_area='box1_cu'))
box2_cu = widgets.Text(layout=widgets.Layout(width='150px', grid_area='box2_cu'))
box3_cu = widgets.Text(layout=widgets.Layout(width='150px', grid_area='box3_cu'))

# Widgets are displayed in a grid
grid_cu = widgets.GridBox(children=[box1_cu, cond1_cu, box2_cu, cond2_cu, box3_cu, cond3_cu],
            layout=widgets.Layout(
            width='550px',
            grid_template_rows='auto auto auto',
            grid_template_columns='150px 200px 200px',
            grid_template_areas='''
            "box1_cu cond1_cu"
            "box2_cu cond2_cu"
            "box3_cu cond3_cu"
            ''')
       )

# This widget contains the three previously defined widgets, side by side.
horiz_cu = widgets.HBox(children=[name_cu, grid_cu, outgraph_cu],
                    layout=widgets.Layout(width='100%', align_items='center', justify_content='center'))

def graph_cu_cond1(box):
    values = np.linspace(0,100,50)
    boxin = []
    for t in values:
        boxin.append(eval(box.value))
    ax_cu.plot(values, boxin, 'k-', label="My graph ($0\leq t\leq 100$)")
    ax_cu.legend()
    with outgraph_cu:
        clear_output(wait=True)
        display(fig_cu)
        
def graph_cu_cond2(box):
    values = np.linspace(100,300,50)
    boxin = []
    for t in values:
        boxin.append(eval(box.value))
    ax_cu.plot(values, boxin, 'k-', label="My graph ($100<t\leq 300$)")
    ax_cu.legend()
    with outgraph_cu:
        clear_output(wait=True)
        display(fig_cu)

def graph_cu_cond3(box):
    values = np.linspace(300,500,50)
    boxin = []
    for t in values:
        boxin.append(eval(box.value))
    ax_cu.plot(values, boxin, 'k-', label="My graph ($t > 500$)")
    ax_cu.legend()
    with outgraph_cu:
        clear_output(wait=True)
        display(fig_cu)

display(horiz_cu)
box1_cu.on_submit(graph_cu_cond1)
box2_cu.on_submit(graph_cu_cond2)
box3_cu.on_submit(graph_cu_cond3)

clear_button_cu = widgets.Button(
                description='Clear graph',
                disabled=False,
                button_style='',
                tooltip='Clear graph',
                layout=widgets.Layout(width='200px')
                )

def clear_graph_cu(clear):
    with outgraph_cu:
        clear_output(wait=True)
        ax_cu.clear()
        start_fig_cu(fig_cu, ax_cu, outgraph_cu)

display(clear_button_cu)
clear_button_cu.on_click(clear_graph_cu)

For CellUniverse:

Button(description='Clear graph', layout=Layout(width='200px'), style=ButtonStyle(), tooltip='Clear graph')

Now, you may have seen that the laws for W(t) and U(t) are:

In [87]:
out_w = widgets.Output()
out_v = widgets.Output()
with out_w:
    display(Latex(r'$$W(t) = \begin{cases} 35, & t \leq 120\\ 35+0.1(t-120), & t> 120\end{cases}$$'))
with out_v:
    display(Latex(r'$$U(t) = \begin{cases} 30, & t\leq 100\\30+0.15(t-100), &100<t\leq 300\\ 30 + 200(0.15) + 0.05(t-300), & t>300.\end{cases}$$'))
accordion = widgets.Accordion(children=[out_w, out_v], selected_index = None)
accordion.set_title(0, 'PhoneWorld:')
accordion.set_title(1, 'CellUniverse:')
display(accordion)

Accordion(children=(Output(), Output()), selected_index=None, _titles={'0': 'PhoneWorld:', '1': 'CellUniverse:…

Finally, to find the intersection points between the two curves, we can see that for the leftmost point, we are in the interval where $100<t<300$, and so we will use the second law for $W$ and the second law for $U$:

\begin{align*}
    35+0.1(t-120) &= 30+0.15(t-100)\\
    35 + 0.1t - 12 &= 30+0.15t - 15\\
    0.1t-0.15t &= 30-15-35+12\\
    -0.05t &= -8
\end{align*}
and so,
$$t = \frac{800}{5} = 160.$$

In fact, we can check that this is true: $W(160)$ is 

In [94]:
phoneworld(160)

39.0

and $U(160)$ is 

In [95]:
celluniverse(160)

39.0

Next, we have a point where $400<t<500$, and so we must have

\begin{align*}
    35+0.1(t-120) &= 30 + 200(0.15) + 0.05(t-300)\\
    35 + 0.1t - 12 &= 30 + 30 + 0.05t - 15\\
    0.1t-0.05t &= 30+30-15-35+12\\
    0.05t &= 22
\end{align*}
and so,
$$t = \frac{2200}{5} = 440.$$

Indeed, $W(440)$ is 

In [93]:
phoneworld(440)

67.0

and $U(440)$ is 

In [91]:
celluniverse(440)

67.0

## Answer

For PhoneWorld to be the best option, our monthly usage must be between 160 and 440 minutes.